In [4]:
from glob import glob
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import shutil
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import time
import h5py
from matplotlib import pyplot as plt
%matplotlib inline

In [5]:

def massageDatatoPNG(): 
    folders = ["braintumourDataset/1_766" ,
           "braintumourDataset/767_1532", 
           "braintumourDataset/1533_2298", 
           "braintumourDataset/2299_3064"]


    #Some bookkeeping as our images are split up
    current_dir = os.getcwd()
    img_dir_0 = os.path.join(current_dir, folders[0])
    img_dir_1 = os.path.join(current_dir, folders[1])
    img_dir_2 = os.path.join(current_dir, folders[2])
    img_dir_3 = os.path.join(current_dir, folders[3])

    #Load image paths
    img_files_0 = [os.path.join(img_dir_0,f) for f in os.listdir(img_dir_0) if (f.endswith('.mat'))]
    img_files_1 = [os.path.join(img_dir_1,f) for f in os.listdir(img_dir_1) if (f.endswith('.mat'))]
    img_files_2 = [os.path.join(img_dir_2,f) for f in os.listdir(img_dir_2) if (f.endswith('.mat'))]
    img_files_3 = [os.path.join(img_dir_3,f) for f in os.listdir(img_dir_3) if (f.endswith('.mat'))]

    img_files = img_files_0 + img_files_1 + img_files_2 + img_files_3
    
    ##Load the images
    imgs =[]
    label=[]

    for f in img_files:
        g = h5py.File(f)
        struArray = g[list(g.keys())[0]]
        imgs.append(struArray['image'][:])
        label.append(struArray['label'][:])
        
#     Convert to jpg images
    directory = os.path.join(current_dir, "ProcessedImages/")
    if not os.path.exists(directory):
        os.makedirs(directory)

    for i in range(len(imgs)): 
        Image.fromarray(np.uint8(imgs[i]), 'L').save("ProcessedImages/" + str(i)+ ".jpg")
        
    return imgs, label

In [6]:
imgs, labels = massageDatatoPNG()


In [8]:
def splitData(ptrain, pval, imSize, label): 
    #Define Sizes
    trainSize = int(np.floor(ptrain*imSize))
    validationSize = int(np.floor(pval*imSize))
    testSize = imSize-trainSize-validationSize

    trainPath = np.empty((trainSize, 2),dtype=object)
    validationPath = np.empty((validationSize,2),dtype=object)
    testPath = np.empty((testSize,2), dtype=object)
    
    for i in range (trainSize): 
        trainPath[i][0] = "ProcessedImages/"+ str(i) + ".jpg"
        trainPath[i][1] = label[i]

    for i in range (validationSize): 
        validationPath[i][0] = "ProcessedImages/"+ str(i + trainSize) + ".jpg"
        validationPath[i][1] = label[i+ trainSize]

    for i in range (testSize): 
        testPath[i][0] = "ProcessedImages/"+ str(i +trainSize+validationSize) + ".jpg"
        testPath[i][1] = label[i +trainSize+validationSize]
    return trainPath, validationPath, testPath

        
    

In [9]:
trainPath, validationPath, testPath = splitData(0.7,0.2,len(imgs), labels)


